# Importing shapefiles into GOSTnets

This is for the scenario if you have a Shapefile of OSM roads. 

In [1]:
import os, sys, importlib

import pandas as pd
import geopandas as gpd
import networkx as nx

from shapely.wkt import loads
from shapely.geometry import Point
from random import sample 

## Your GOSTnets path could be different
#sys.path.append(r"C:\Work\Code\GOSTnets")
sys.path.append("../../../GOSTNets")
import GOSTnets as gn

In [2]:
from GOSTnets.load_osm import *

In [3]:
# turn on extention to auto reload module changes
%load_ext autoreload
%autoreload 2

In [7]:
# shapefile is in path
input_shapefile = './input_folder/cxb_roads_edit.shp'
os.path.isfile(input_shapefile) 

True

### Creating and OSM_to_network object. Internally this will fetch roads from the shapefile and save it to the objects roads_raw property, which is a GeoPandasDataframe

In [9]:
cxb_roads = OSM_to_network(input_shapefile)

In [10]:
cxb_roads.roads_raw

,stnode,endnode,length,infra_type,osm_id,unique_id,key,Type,id,geometry
0,0,4690,0.128829,unclassified,644427970,0,edge_9669,legitimate,9669,"LINESTRING (92.32052 20.82725, 92.32021 20.827..."
1,1,4286,0.466935,unclassified,527438783,1,None,legitimate,5076,"LINESTRING (92.08362 21.43088, 92.08360 21.431..."
2,2,9006,0.133592,residential,527803779,2,edge_5595,legitimate,5595,"LINESTRING (92.13161 21.29078, 92.13145 21.290..."
3,7,3859,0.036216,track,537913478,5,edge_7883,legitimate,7883,"LINESTRING (92.14399 21.17240, 92.14384 21.172..."
4,9,4286,0.128266,unclassified,530360574,7,edge_6859,legitimate,6859,"LINESTRING (92.08659 21.43367, 92.08670 21.433..."
...,...,...,...,...,...,...,...,...,...,...
18194,9541,855,0.072594,residential,553983671,18199,edge_8254,legitimate,8254,"LINESTRING (92.16359 21.20490, 92.16370 21.204..."
18195,0,3763,0.098504,track,555571341,18203,edge_8363,origin_destruction,8363,"LINESTRING (92.15784 21.21819, 92.15784 21.218..."
18196,0,880,0.026264,road,530494978,18204,None,origin_destruction,7018,"LINESTRING (91.99631 21.49519, 91.99632 21.495..."
18197,4139,6238,1.207766,residential,532124941,7919,edge_7679,legitimate,7679,"LINESTRING (92.05785 21.48789, 92.05786 21.487..."


In [22]:
# cxb_roads.roads_raw.infra_type.value_counts()
len(cxb_roads.roads_raw)

18199

In [12]:
# In this case we will also capture residential and unclassified roads, which aren't part of the defaults
accepted_road_types = ['residential', 'unclassified', 'track', 'service', 'tertiary', 'road', 'secondary', 'primary', 'trunk','motorway','motorway_link','primary_link', 'trunk_link', 'tertiary_link', 'secondary_link']

In [13]:
cxb_roads.filterRoads(acceptedRoads = accepted_road_types)

In [21]:
len(cxb_roads.roads_raw)
# can tell that the filter did not reduce any of the road count

18199

In [15]:
cxb_roads.generateRoadsGDF(verbose = False)

In [16]:
cxb_roads.roadsGPD

,Wkt,osm_id,infra_type,id,key,length,u,v
0,"LINESTRING (92.32052 20.82725, 92.32021 20.827...",0,unclassified,1,edge_1,0.128829,"(92.3205152, 20.827246)","(92.3193651, 20.8269117)"
1,"LINESTRING (92.08362 21.43088, 92.08360 21.431...",1,unclassified,2,edge_2,0.466935,"(92.0836204, 21.4308841)","(92.0868579, 21.4326044)"
2,"LINESTRING (92.13161 21.29078, 92.13145 21.290...",2,residential,3,edge_3,0.133592,"(92.1316132, 21.2907826)","(92.1314013, 21.291856)"
3,"LINESTRING (92.14399 21.17240, 92.14384 21.172...",5,track,4,edge_4,0.036216,"(92.1439931, 21.1723959)","(92.1437672, 21.172236)"
4,"LINESTRING (92.08659 21.43367, 92.08670 21.433...",7,unclassified,5,edge_5,0.128266,"(92.0865921, 21.4336693)","(92.0868579, 21.4326044)"
...,...,...,...,...,...,...,...,...
18272,"LINESTRING (92.16359 21.20490, 92.16370 21.204...",18199,residential,18273,edge_18273,0.072594,"(92.1635891, 21.2048994)","(92.1642617, 21.2047306)"
18273,"LINESTRING (92.15784 21.21819, 92.15784 21.218...",18203,track,18274,edge_18274,0.098092,"(92.15783655000003, 21.21818705000011)","(92.1581028, 21.2189323)"
18274,"LINESTRING (91.99631 21.49519, 91.99632 21.495...",18204,road,18275,edge_18275,0.026348,"(91.9963117, 21.4951869)","(91.99632390000001, 21.49526230000004)"
18275,"LINESTRING (92.05785 21.48789, 92.05786 21.487...",7919,residential,18276,edge_18276,0.445815,"(92.05784774320709, 21.487890336352677)","(92.0613313, 21.4893066)"


In [17]:
cxb_roads.initialReadIn()

In [18]:
gn.save(cxb_roads.network,'cxb_roads_unclean','./')